In [17]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [18]:
video_path = r'D:\MAPUA\Thesis\Video_Sample1\Video1.mp4'
output_folder = r'D:\MAPUA\Thesis\OutputVideo_Sample1'

def extract_landmarks_from_video(video_path):
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Read the video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create an output folder with the same name as the video file
    output_folder = os.path.splitext(os.path.basename(video_path))[0]
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each frame
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert BGR frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame with MediaPipe Pose
        results = pose.process(frame_rgb)

        # Draw landmarks on the frame
        if results.pose_landmarks:
            height, width, _ = frame.shape
            for landmark in results.pose_landmarks.landmark:
                x, y = int(landmark.x * width), int(landmark.y * height)
                cv2.circle(frame, (x, y), radius=2, color=(0, 255, 0), thickness=2)

        # Save the frame with landmarks as an image
        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        annotated_frame = frame.copy()  # Create a copy of the frame for drawing landmarks
        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                                      mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0),
                                                                                             thickness=2,
                                                                                             circle_radius=2))
        cv2.imwrite(frame_path, annotated_frame)

        # Display the frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

    print(f"Images saved to {output_folder} folder.")

extract_landmarks_from_video(video_path)
